In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import scipy.stats as scistats
import pandas as pd
import seaborn as sns
import numpy as np
import statistics
import math

In [2]:
all_samples = sc.read('all_groups.h5ad')

In [3]:
save_path = 'figures'

In [4]:
sc.set_figure_params(dpi=200)

In [5]:
new_tp = []
for tp in all_samples.obs['timepoint']:
    if tp == 'week2':
        new_tp.append('Week 2')
    elif tp == 'week4':
        new_tp.append('Week 4')
    elif tp == 'week6':
        new_tp.append('Week 6')
all_samples.obs['new_timepoint'] = new_tp

In [6]:
monos = all_samples[(all_samples.obs['Cell type'] == 'Monocyte')]
b_cells = all_samples[(all_samples.obs['Cell type'] == 'B cell')]

# 3a: General DEG across all cell types

In [ ]:
fig, ax = plt.subplots(figsize=(4, 10))
sc.tl.rank_genes_groups(all_samples, groupby='new_timepoint', n_genes=10, method='wilcoxon')
ret = sc.pl.rank_genes_groups_stacked_violin(all_samples, groupby='new_timepoint', n_genes=15,
                                             categories_order=['Week 2', 'Week 4', 'Week 6'], swap_axes=True,
                                             dendrogram=False, save=False, ax=ax)
fig.savefig(f"{save_path}/DEG_all_patients_Wilcoxon.pdf", bbox_inches='tight')

# 3b: HIF1A UMAP

In [ ]:
sc.pl.umap(all_samples, color='HIF1A', size=1, save="HIF1A_umap.pdf")

# 3c: HIF1A positive cells per cell type in %

In [9]:
cell_type_list = ['CD4 T cell', 'CD8 T cell', 'gdT cell', 'NK T cell', 'NK cell', 'Monocyte', 'DC', 'B cell']

In [10]:
genes = all_samples.var_names
genes = list(genes)


def cell_type_bar_gene_expr_percentage(adata, gene, cs):
    adata_temp = adata
    adata_temp.obs["value"] = 0
    if cs != 'all':
        adata_temp = adata_temp[(adata_temp.obs['clinical_score'] == cs)]

    x = cell_type_list
    y = []
    for cell_type in cell_type_list:
        adata_ct_temp = adata_temp[(adata_temp.obs['Cell type'] == cell_type)]
        
        positive = adata_ct_temp[adata_ct_temp[:, 'HIF1A'].X > 0]
        total = adata_ct_temp.shape[0]
        percentage = (positive.shape[0] / total) * 100

        print(percentage)
        y.append(percentage)

    plt.rcParams["figure.figsize"] = 4, 4
    bars = plt.bar(x, y)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() - .15, yval + .5, round(yval, 2), rotation=25, fontsize=12)
    #plt.title(f'{gene} positive cells')#; clinical score: {cs}
    plt.xlabel('Cell type')
    plt.ylabel('HIF1A positive cells [%]')
    plt.ylim(0, 100)
    ax = plt.gca()
    ax.set_xticklabels(labels=cell_type_list, rotation=90)
    plt.savefig(f'{save_path}/Percentage_HIF1A_positive_cells.pdf', bbox_inches='tight')

In [ ]:
cell_type_bar_gene_expr_percentage(all_samples, 'HIF1A', 'all')

# 3f: Correlation: HIF1A & cs

In [12]:
def correlation_logfc_cs(adata, gene, cell_type):
    if cell_type != 'all':
        adata = adata[(adata.obs['Cell type'] == cell_type)]
    pos_gene = -1
    for i, gn in enumerate(adata.var_names):
        if gn == gene:
            pos_gene = i

    plt.figure(figsize=(4, 4))
    logfc_values_week4, logfc_values_week6, cs_values = [], [], []
    for cs in list(set(adata.obs['clinical_score'])):
        adata_cs = adata[(adata.obs['clinical_score'] == cs)]
        week2 = adata_cs[(adata_cs.obs['timepoint'] == 'week2')]
        week4 = adata_cs[(adata_cs.obs['timepoint'] == 'week4')]
        week6 = adata_cs[(adata_cs.obs['timepoint'] == 'week6')]

        #calculate mean of normalized (NOT log-normalized) counts
        mean_week2 = statistics.mean(week2.layers['two_batch_correction'][:, pos_gene])
        mean_week4 = statistics.mean(week4.layers['two_batch_correction'][:, pos_gene])
        mean_week6 = statistics.mean(week6.layers['two_batch_correction'][:, pos_gene])
        lfc_week2vs4 = mean_week2 / mean_week4
        lfc_week2vs6 = mean_week2 / mean_week6

        logfc_values_week4.append(math.log2(lfc_week2vs4))
        logfc_values_week6.append(math.log2(lfc_week2vs6))
        cs_values.append(cs)

    x = np.array(cs_values)
    y = np.array(logfc_values_week4)
    plt.scatter(x, y, color='blue')
    plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)), color='blue', label="Week 2 vs 4")
    y = np.array(logfc_values_week6)
    plt.scatter(x, y, color='red')
    plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)), color='red', label="Week 2 vs 6")

    x = pd.Series(cs_values)
    y = pd.Series(logfc_values_week4)
    correlation = y.corr(x)
    print(f"week 2 vs 4:  {correlation}")
    plt.text(18, -0.5, f'PCC: {round(correlation, 2)}', color='blue')
    y = pd.Series(logfc_values_week6)
    correlation = y.corr(x)
    print(f"week 2 vs 6:  {correlation}")
    plt.text(18, -0.62, f'PCC: {round(correlation, 2)}', color='red')

    plt.legend()
    plt.xlabel('Clinical score')
    plt.ylabel('Log2 fold change')
    plt.savefig(f'{save_path}/{cell_type}_HIF1A_cs_correlation.pdf', bbox_inches='tight')

In [ ]:
for cell_type in cell_type_list:
    correlation_logfc_cs(all_samples, 'HIF1A', cell_type)

# 3d: HIF1A in other cell types (stacked violin with p-values)

In [14]:
def violin_with_p_gene_for_cs(adata_temp, clinical_score, gene, cell_type, axarr, gene_nr, custom_top):
    plt.sca(axarr[gene_nr])
    if clinical_score != 'all':
        adata_temp_cs = adata_temp[adata_temp.obs['clinical_score'] == clinical_score]
    else:
        adata_temp_cs = adata_temp
    adata_temp_cs.obs["value"] = 0
    position_gene = -1
    for i, gn in enumerate(adata_temp_cs.var_names):
        if gn == gene:
            position_gene = i

    if cell_type != 'all cell':
        adata_cs_type = adata_temp_cs[adata_temp_cs.obs['Cell type'] == cell_type]
    else:
        adata_cs_type = adata_temp_cs
    adata_cs_type.obs["value"] = 0

    sc.settings.verbosity = 0

    df = pd.DataFrame({"Timepoint": adata_cs_type.obs['new_timepoint'],
                       "expression": adata_cs_type.layers['two_batch_correction'][:, position_gene]})
    ax = sns.violinplot(data=df, y="expression", x="Timepoint", order=['Week 2', 'Week 4', 'Week 6'])

    if clinical_score == 'all':
        if cell_type != 'all cell':
            ax.set_ylabel(cell_type)
        else:
            ax.set_ylabel(gene)
    else:
        ax.set_ylabel(f'c.s. {clinical_score}')

    week2 = adata_cs_type[adata_cs_type.obs['timepoint'] == 'week2']
    week4 = adata_cs_type[adata_cs_type.obs['timepoint'] == 'week4']
    week6 = adata_cs_type[adata_cs_type.obs['timepoint'] == 'week6']

    p_adj_week2vs4 = scistats.ranksums(week2.layers['two_batch_correction'][:, position_gene],
                                       week4.layers['two_batch_correction'][:, position_gene],
                                       alternative='greater')[1]
    p_adj_week2vs6 = scistats.ranksums(week2.layers['two_batch_correction'][:, position_gene],
                                       week6.layers['two_batch_correction'][:, position_gene],
                                       alternative='greater')[1]
    p_adj_week4vs6 = scistats.ranksums(week4.layers['two_batch_correction'][:, position_gene],
                                       week6.layers['two_batch_correction'][:, position_gene],
                                       alternative='greater')[1]

    top = np.ceil(np.max(adata_cs_type.layers['two_batch_correction'][:, position_gene]))

    ax.annotate("", xy=(0.15, top), xytext=(0.85, top),
                arrowprops={'arrowstyle': '-'}, va='center')
    ax.annotate("", xy=(1.15, top), xytext=(1.85, top),
                arrowprops={'arrowstyle': '-'}, va='center')
    ax.annotate("", xy=(0.15, top + 1.6), xytext=(1.85, top + 1.6),
                arrowprops={'arrowstyle': '-'}, va='center')

    if p_adj_week2vs6 == 0 or p_adj_week2vs6 >= 1:
        ax.annotate(min(int(p_adj_week2vs6), 1), xy=(1, top + 1.75), fontsize=9)
    else:
        ax.annotate('{:.2e}'.format(p_adj_week2vs6), xy=(0.8, top + 1.75), fontsize=9)

    if p_adj_week2vs4 == 0 or p_adj_week2vs4 >= 1:
        ax.annotate(min(int(p_adj_week2vs4), 1), xy=(0.4, top + 0.15), fontsize=9)
    else:
        ax.annotate('{:.2e}'.format(p_adj_week2vs4), xy=(0.2, top + 0.15), fontsize=9)
    if p_adj_week4vs6 == 0 or p_adj_week4vs6 >= 1:
        ax.annotate(min(int(p_adj_week4vs6), 1), xy=(1.4, top + 0.15), fontsize=9)
    else:
        ax.annotate('{:.2e}'.format(p_adj_week4vs6), xy=(1.2, top + 0.15), fontsize=9)

In [15]:
def violin_stacked_by_cell_type(gene, adata, custom_top):
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = 3, (8 * 1.5)
    gs = fig.add_gridspec(8, hspace=0)
    axs = gs.subplots(sharex=True, sharey=True)

    cell_type_list = ['CD4 T cell', 'CD8 T cell', 'gdT cell', 'NK T cell', 'NK cell', 'Monocyte', 'DC', 'B cell']

    for i, cell_type in enumerate(cell_type_list):
        violin_with_p_gene_for_cs(adata, clinical_score='all', gene=gene, cell_type=cell_type, axarr=axs, gene_nr=i,
                                  custom_top=custom_top)

    # Hide x labels and tick labels for all but bottom plot.
    for ax in axs:
        ax.label_outer()

    plt.xlabel("")

    plt.savefig(f'{save_path}/{gene}_stacked_by_cell_type_greater_V2.pdf', bbox_inches='tight')

In [ ]:
violin_stacked_by_cell_type('HIF1A', all_samples, 11)

# 3e: Violin stacked by clinical score

In [17]:
def violin_with_p_gene(adata_temp, clinical_score, gene, cell_type, axarr, gene_nr, custom_top):
    plt.sca(axarr[gene_nr])
    if clinical_score != 'all':
        adata_temp_cs = adata_temp[adata_temp.obs['clinical_score'] == clinical_score]
    else:
        adata_temp_cs = adata_temp
    adata_temp_cs.obs["value"] = 0
    position_gene = -1
    for i, gn in enumerate(adata_temp_cs.var_names):
        if gn == gene:
            position_gene = i

    if cell_type != 'all cell':
        adata_cs_type = adata_temp_cs[adata_temp_cs.obs['Cell type'] == cell_type]
    else:
        adata_cs_type = adata_temp_cs
    adata_cs_type.obs["value"] = 0

    sc.settings.verbosity = 0

    df = pd.DataFrame({"Timepoint": adata_cs_type.obs['new_timepoint'],
                       "expression": adata_cs_type.layers['two_batch_correction'][:, position_gene]})
    ax = sns.violinplot(data=df, y="expression", x="Timepoint", order=['Week 2', 'Week 4', 'Week 6'])
    if clinical_score == 'all':
        ax.set_ylabel(gene)
    else:
        ax.set_ylabel(f'c.s. {clinical_score}')

    week2 = adata_cs_type[adata_cs_type.obs['timepoint'] == 'week2']
    week4 = adata_cs_type[adata_cs_type.obs['timepoint'] == 'week4']
    week6 = adata_cs_type[adata_cs_type.obs['timepoint'] == 'week6']

    p_adj_week2vs4 = scistats.ranksums(week2.layers['two_batch_correction'][:, position_gene],
                                       week4.layers['two_batch_correction'][:, position_gene],
                                       alternative='greater')[1]
    p_adj_week2vs6 = scistats.ranksums(week2.layers['two_batch_correction'][:, position_gene],
                                       week6.layers['two_batch_correction'][:, position_gene],
                                       alternative='greater')[1]
    p_adj_week4vs6 = scistats.ranksums(week4.layers['two_batch_correction'][:, position_gene],
                                       week6.layers['two_batch_correction'][:, position_gene],
                                       alternative='greater')[1]

    top = np.ceil(np.max(adata_cs_type.layers['two_batch_correction'][:, position_gene]))

    ax.annotate("", xy=(0.15, top), xytext=(0.85, top),
                arrowprops={'arrowstyle': '-'}, va='center')
    ax.annotate("", xy=(1.15, top), xytext=(1.85, top),
                arrowprops={'arrowstyle': '-'}, va='center')
    ax.annotate("", xy=(0.15, top + 1.25), xytext=(1.85, top + 1.25),
                arrowprops={'arrowstyle': '-'}, va='center')

    if p_adj_week2vs6 == 0 or p_adj_week2vs6 >= 1:
        ax.annotate(min(int(p_adj_week2vs6), 1), xy=(1, top + 1.4), fontsize=9)
    else:
        ax.annotate('{:.2e}'.format(p_adj_week2vs6), xy=(0.8, top + 1.4), fontsize=9)

    if p_adj_week2vs4 == 0 or p_adj_week2vs4 >= 1:
        ax.annotate(min(int(p_adj_week2vs4), 1), xy=(0.4, top + 0.15), fontsize=9)
    else:
        ax.annotate('{:.2e}'.format(p_adj_week2vs4), xy=(0.2, top + 0.15), fontsize=9)
    if p_adj_week4vs6 == 0 or p_adj_week4vs6 >= 1:
        ax.annotate(min(int(p_adj_week4vs6), 1), xy=(1.4, top + 0.15), fontsize=9)
    else:
        ax.annotate('{:.2e}'.format(p_adj_week4vs6), xy=(1.2, top + 0.15), fontsize=9)

    ax.set_ylim(top=custom_top)

In [18]:
def violin_stacked_by_cs(gene, adata, custom_top, cell_type):
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = 3, (6 * 2)
    gs = fig.add_gridspec(6, hspace=0)
    axs = gs.subplots(sharex=True, sharey=True)
    for i, cs in enumerate([0, 11, 14, 17, 19, 26]):
        violin_with_p_gene(adata, clinical_score=cs, gene=gene, cell_type='all cell', axarr=axs, gene_nr=i,
                           custom_top=custom_top)

    # Hide x labels and tick labels for all but bottom plot.
    for ax in axs:
        ax.label_outer()

    plt.xlabel("")

    plt.savefig(f'{save_path}/{cell_type}_{gene}_stacked_by_cs.pdf', bbox_inches='tight')

In [ ]:
violin_stacked_by_cs('HIF1A', monos, 11, "Monocytes")

In [ ]:
violin_stacked_by_cs('HIF1A', b_cells, 11, "B cells")

# 3g: Coexpression of HIF1A with other genes

In [21]:
all_genes_df = pd.DataFrame(data=all_samples.layers['two_batch_correction'], columns=all_samples.var_names)

In [22]:
corr_square = all_genes_df.corr()

In [23]:
spearman_corr_square = all_genes_df.corr(method='spearman')
spearman_HIF1A_corr_square = spearman_corr_square[['HIF1A']]

In [ ]:
# top 15 genes correlated to HIF1A
spearman_big_corrs = spearman_HIF1A_corr_square[
    (spearman_HIF1A_corr_square['HIF1A'] > 0.39) | (spearman_HIF1A_corr_square['HIF1A'] < -0.39)]
spearman_big_corrs

In [ ]:
fig, ax = plt.subplots(figsize=(13, 1))
sns.heatmap(spearman_big_corrs.transpose(), cmap="Blues", annot=True)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.xlabel('')
plt.ylabel('')
plt.savefig(f'{save_path}/HIF1A_Spearman_heatmap.pdf', bbox_inches='tight')